In [1]:
import os
import re
import ast
import math
import nltk
import spacy
import torch
import string
import random
import contractions
import numpy as np
import pandas as pd
import seaborn as sns
import scattertext as st
import plotly.express as px
import matplotlib.pyplot as plt
import dataset_utils
import absa_utils

from collections import Counter
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer

from pyabsa import ATEPCCheckpointManager
from pyabsa import available_checkpoints

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

No CUDA GPU found in your device
[2024-02-18 10:53:43] (2.4.0) PyABSA(2.4.0): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






c:\Users\raffa\anaconda3\envs\SCMS\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\raffa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raffa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\raffa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Setting seed

In [2]:
# Set the random seed for reproducibility.
random_seed = 1
torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

# Constants

In [3]:
PREPROCESSED_DATASET_PATH = "./dataset/dataset.csv"
ABSA_REVIEWS_PATH = "./dataset/absa_reviews.csv"
ABSA_DATASET_PATH = "./dataset/absa_dataset.csv"
DATASET_WITH_SENTIMENT_PATH = "./dataset/dataset_with_sentiment.csv"

# Plots

In [4]:
dataset = pd.read_csv(ABSA_DATASET_PATH)

C:\Users\raffa\AppData\Local\Temp\ipykernel_25836\1459262514.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(ABSA_DATASET_PATH)


In [5]:
absa_utils.plot_most_frequent_aspects(dataset, k=20)

C:\Users\raffa\AppData\Roaming\Python\Python311\site-packages\kaleido\scopes\base.py:185: DeprecationWarning:

setDaemon() is deprecated, set the daemon attribute instead



In [6]:
aspects_df, filtered_aspects_df = absa_utils.plot_top_and_bottom_aspects(dataset, k=20)

First quartile: 1.0
Second quartile: 1.0
Third quartile: 3.0
Percentile 95: 25.0


In [7]:
filtered_aspects_df

,Frequency,Sum,Mean,Std,Sentiments
kind bars,27,27,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
staple,46,46,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
virgin coconut oil,26,26,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
concept,29,29,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
snack bar,120,120,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...
pricey,27,-25,-0.925926,0.377705,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1..."
flavorless,85,-79,-0.929412,0.369044,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
sodium content,37,-35,-0.945946,0.324324,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1..."
advertising,40,-38,-0.950000,0.312250,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [8]:
filtered_aspects_df.head(50)

,Frequency,Sum,Mean,Std,Sentiments
kind bars,27,27,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
staple,46,46,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
virgin coconut oil,26,26,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
concept,29,29,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
snack bar,120,120,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
ships,106,106,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
portability,26,26,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
memories,36,36,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
shopping experience,35,35,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
transaction,208,208,1.000000,0.000000,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [9]:
filtered_aspects_df.tail(50)

,Frequency,Sum,Mean,Std,Sentiments
sugar content,147,-60,-0.408163,0.901662,"[-1, 0, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
hulls,118,-49,-0.415254,0.905036,"[-1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1,..."
formula,149,-64,-0.429530,0.884278,"[-1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -..."
tops,50,-22,-0.440000,0.875443,"[1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1..."
sugar alcohol,80,-36,-0.450000,0.820061,"[-1, -1, -1, -1, 0, -1, 1, -1, -1, -1, -1, 1, ..."
artificial sweetener,75,-34,-0.453333,0.883830,"[-1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1,..."
label,214,-98,-0.457944,0.829146,"[-1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1,..."
high fructose corn syrup,28,-13,-0.464286,0.778375,"[1, -1, -1, -1, 0, -1, -1, -1, 1, 0, -1, -1, -..."
crumbs,30,-14,-0.466667,0.884433,"[1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, ..."
expense,27,-13,-0.481481,0.876456,"[-1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -..."


In [10]:
top_aspect = 'price'

In [11]:
absa_utils.plot_aspect_sentiment_in_time(dataset, top_aspect)

In [5]:
absa_utils.plot_price_information(dataset)

Aspect:  bargain
Frequency:  72
Sum:  72
Mean:  1.0
Std:  0.0

Aspect:  convenience
Frequency:  242
Sum:  220
Mean:  0.9090909090909091
Std:  0.3962635403218795

Aspect:  cost
Frequency:  1735
Sum:  7
Mean:  0.004034582132564841
Std:  0.9872302120576868

Aspect:  costs
Frequency:  152
Sum:  -33
Mean:  -0.21710526315789475
Std:  0.9659857465842228

Aspect:  deal
Frequency:  1365
Sum:  1320
Mean:  0.967032967032967
Std:  0.2532086885136759

Aspect:  discount
Frequency:  25
Sum:  -3
Mean:  -0.12
Std:  0.9516301802696256

Aspect:  expensive
Frequency:  206
Sum:  -196
Mean:  -0.9514563106796117
Std:  0.30778383464363807

Aspect:  money
Frequency:  1060
Sum:  -141
Mean:  -0.1330188679245283
Std:  0.9849070158307742

Aspect:  price
Frequency:  40398
Sum:  28087
Mean:  0.695257190950047
Std:  0.7053948132859176

Aspect:  priced
Frequency:  2252
Sum:  1065
Mean:  0.47291296625222023
Std:  0.8798483265214951

Aspect:  prices
Frequency:  1067
Sum:  271
Mean:  0.2539831302717901
Std:  0.9515786576